<a href="https://colab.research.google.com/github/Hemila-cloud/smart-waste-classifier/blob/main/train_waste_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Uploading Trash Dataset**

In [15]:
from google.colab import files
uploaded = files.upload()  # Upload your ZIP file here


Saving dataset-resized.zip.zip to dataset-resized.zip.zip


In [17]:
import zipfile
import os

with zipfile.ZipFile("dataset-resized.zip.zip", 'r') as zip_ref:
    zip_ref.extractall()
print("✅ Dataset extracted")


✅ Dataset extracted


# **Install Required Libraries**

In [18]:
!pip install tensorflow pillow numpy


# **Train the Model in Colab**

In [20]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

# Constants
DATASET_DIR = "dataset-resized.zip"
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 16
EPOCHS = 10

# Load and preprocess data
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
)

train_gen = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# Load base model
base_model = MobileNetV2(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
base_model.trainable = False

# Add classifier
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(train_gen.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
checkpoint = ModelCheckpoint("waste_model.h5", monitor='val_accuracy', save_best_only=True)
history = model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS, callbacks=[checkpoint])

print("✅ Training complete. Model saved as waste_model.h5")


Found 2024 images belonging to 6 classes.
Found 503 images belonging to 6 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.


Epoch 1/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 789ms/step - accuracy: 0.4039 - loss: 1.5264

127/127 ━━━━━━━━━━━━━━━━━━━━ 135s 1s/step - accuracy: 0.4050 - loss: 1.5242 - val_accuracy: 0.6143 - val_loss: 1.0223
Epoch 2/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 772ms/step - accuracy: 0.7233 - loss: 0.7873

127/127 ━━━━━━━━━━━━━━━━━━━━ 123s 969ms/step - accuracy: 0.7235 - loss: 0.7869 - val_accuracy: 0.6839 - val_loss: 0.8313
Epoch 3/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 767ms/step - accuracy: 0.8029 - loss: 0.5867

127/127 ━━━━━━━━━━━━━━━━━━━━ 123s 969ms/step - accuracy: 0.8029 - loss: 0.5867 - val_accuracy: 0.7137 - val_loss: 0.7665
Epoch 4/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 121s 954ms/step - accuracy: 0.8259 - loss: 0.5325 - val_accuracy: 0.7117 - val_loss: 0.7591
Epoch 5/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 770ms/step - accuracy: 0.8416 - loss: 0.4666

127/127 ━━━━━━━━━━━━━━━━━━━━ 139s 1s/step - accuracy: 0.8417 - loss: 0.4664 - val_accuracy: 0.7416 - val_loss: 0.6745
Epoch 6/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 122s 943ms/step - accuracy: 0.8735 - loss: 0.4144 - val_accuracy: 0.7237 - val_loss: 0.7175
Epoch 7/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 816ms/step - accuracy: 0.8936 - loss: 0.3451

127/127 ━━━━━━━━━━━━━━━━━━━━ 130s 1s/step - accuracy: 0.8935 - loss: 0.3453 - val_accuracy: 0.7495 - val_loss: 0.6687
Epoch 8/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 753ms/step - accuracy: 0.8920 - loss: 0.3356

127/127 ━━━━━━━━━━━━━━━━━━━━ 120s 946ms/step - accuracy: 0.8920 - loss: 0.3355 - val_accuracy: 0.7674 - val_loss: 0.6421
Epoch 9/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 139s 1s/step - accuracy: 0.8980 - loss: 0.3204 - val_accuracy: 0.7435 - val_loss: 0.6687
Epoch 10/10
127/127 ━━━━━━━━━━━━━━━━━━━━ 128s 1s/step - accuracy: 0.9104 - loss: 0.2782 - val_accuracy: 0.7296 - val_loss: 0.6941
✅ Training complete. Model saved as waste_model.h5


# **Test the Model on a New Image**

In [38]:
from google.colab import files
uploaded = files.upload()  # Upload test.jpg or any image


Saving metal2.webp to metal2.webp


In [39]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

# Load saved model
model = load_model("waste_model.h5")

# Class names should match your folders
CLASS_NAMES = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

def classify_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x / 255.0

    preds = model.predict(x)
    idx = np.argmax(preds)
    label = CLASS_NAMES[idx]
    confidence = float(preds[0][idx]) * 100

    return label, confidence

# Replace with your uploaded image name
label, conf = classify_image("metal2.webp")
print("Prediction:", label)
print("Confidence:", f"{conf:.2f}%")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Prediction: metal
Confidence: 73.65%


In [30]:
from google.colab import files
files.download("waste_model.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>